In [42]:
import json
import pandas as pd
import numpy as np
from joblib import load, dump
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score

from bayes_opt import BayesianOptimization

from config import settings
from process_and_ml.where_is import WhereIs
from process_and_ml import load_azdias, load_test_file, load_customers, CleanUp
from process_and_ml import fs_pipeline, confirm_equal_columns_dataframe
from process_and_ml.pipeline import compute_metrics

In [2]:
where_is = WhereIs(False)


In [3]:
all_paths = where_is.get_paths_list

In [4]:
train_df = pd.read_csv(all_paths[18], sep=';')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train_df = train_df.rename(columns={'CAMEO_INTL_2015': 'CAMEO_DEUINTL_2015'})

In [6]:
response = train_df['RESPONSE']

In [7]:
train_df.drop(['RESPONSE'], axis=1, inplace=True)


In [8]:
customer_df = load_customers(all_paths[1])

In [9]:
confirm_equal_columns_dataframe(train_df, customer_df)

set()

In [10]:
train_df.CAMEO_DEUG_2015.tail()

42957    3.0
42958    6.0
42959    4.0
42960    8.0
42961    4.0
Name: CAMEO_DEUG_2015, dtype: object

In [18]:
clean_up = CleanUp(all_paths)

In [19]:
dfs_dict = clean_up.pipeline_clean_up(dfs={'train':train_df, 'customers': customer_df},unknowns_df=all_paths[16])


                      Unnamed: 0.1  Value  TestDvC
Attribute                                         
AGER_TYP                         1     -1        2
ALTERSKATEGORIE_GROB             6  -1, 0        2
ANREDE_KZ                       34  -1, 0        2
BALLRAUM                        41     -1        2
BIP_FLAG                        49     -1        2


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [20]:
train_df_clean = dfs_dict['train']

In [21]:
train_df_clean.loc[:, 'CAMEO_DEUG_2015'] = train_df_clean.CAMEO_DEUG_2015.astype('float64')


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [22]:
customer_df.CAMEO_DEUG_2015.dtypes

dtype('float64')

In [23]:
train_df_clean.CAMEO_DEUG_2015.dtypes

dtype('float64')

In [71]:
train_df_clean.loc[:, 'CAMEO_DEUG_2015'] = train_df_clean.CAMEO_DEUG_2015.astype('float64')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [25]:
train_df_clean.CAMEO_DEUINTL_2015.dtypes

dtype('O')

In [26]:
dfs_dict['customers'].CAMEO_DEUINTL_2015.dtypes

dtype('float64')

In [73]:
columns_list = [col for col in train_df.columns if col.startswith('CAMEO')]

In [27]:
train__df_after_pipe = fs_pipeline(train_df_clean)

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/p

Created WEALTH and LIFE_AGE column from 'CAMEO_DEUINTL_2015', original column was droped
Created 'PLZ8_BAUMAX_BUSSINESS' and 'PLZ8_BAUMAX_FAMILY' from 'PLZ8_BAUMAX_FAMILY', original column was droped.
Created 'PRAEGENDE_JUGENDJAHRE_GEN' and 'PRAEGENDE_JUGENDJAHRE_MOV' from 'PRAEGENDE_JUGENDJAHRE', original column was droped.
Working on Binary columns.
Shape Before Remove KBA -> (42962, 276)
Removing KBA05 columns
Get_Dummies with pandas
Get_Dummies with pandas cat column = ['CAMEO_DEU_2015']


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/series.py:4515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [29]:
with open('../data/cleaned_data/azdias_columns.json', 'r') as jsonfile:
    azdias_columns = json.load(jsonfile)


In [30]:
train__df_after_pipe = train__df_after_pipe[azdias_columns['columns']]

In [31]:
unsupervised_pipeline = load('../models/unsupervised_transform_2.joblib')

In [44]:
a = unsupervised_pipeline.steps[0]

In [32]:
train__df_after_pipe = unsupervised_pipeline.transform(train__df_after_pipe)

In [37]:
x_train, x_test, y_train, y_test = train_test_split(train__df_after_pipe, response.values, random_state=settings.RANDOM_STATE, test_size=0.3)

In [38]:
# As wrote on proposal let's start with the more simple model. LogisticRegressionCV

log_reg = LogisticRegressionCV(cv=10)

In [39]:
log_reg.fit(x_train, y_train)


LogisticRegressionCV(cv=10)

In [45]:
compute_metrics(log_reg, x_test, y_test)

Metrics(ACC=0.9866552874544186, AUC=0.5333327389998007)